In [1]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import urllib2
import bs4
import time
import operator
import re 
    
%matplotlib inline
import seaborn as sns

In [37]:
data = pd.read_csv('cleaned_GaCountiesODDeath.csv', na_values=[" ", ""] ,sep=',', index_col=0)
hospital = pd.read_csv('cleaned_GaHospitals.csv', na_values=[" ", ""] ,sep=',', index_col=0)

In [43]:
df.head()

,County,Rural,1yrlag,5yrlag,10yrlag,2016
0,Appling,1,0.000000,0.000000,0,3
1,Atkinson,1,0.000000,0.000000,0,1
2,Bacon,1,0.000000,-1.000000,-1,0
3,Baker,1,-1.000000,0.000000,0,0
4,Banks,1,0.333333,-0.333333,3,4


In [39]:
#data[['Counties','Rural', '2016', '1yrlag', '2yrlag', '10yrlag']]
data['1yrlag']=(data['2016']-data['2015'])/data['2015']
data['5yrlag']=(data['2016']-data['2011'])/data['2011']
data['10yrlag']=(data['2016']-data['2006'])/data['2006']
data=data.replace([np.inf, -np.inf], np.nan)
data = data.fillna(0)

In [41]:
df = data[['County','Rural', '1yrlag', '5yrlag', '10yrlag','2016']].copy()

In [44]:
pd.concat([df, hospital['Count']], axis=1)

,County,Rural,1yrlag,5yrlag,10yrlag,2016,Count
0,Appling,1,0.000000,0.000000,0.000000,3,0
1,Atkinson,1,0.000000,0.000000,0.000000,1,0
2,Bacon,1,0.000000,-1.000000,-1.000000,0,0
3,Baker,1,-1.000000,0.000000,0.000000,0,0
4,Banks,1,0.333333,-0.333333,3.000000,4,2
5,Ben Hill,1,0.000000,0.000000,0.000000,1,0
6,Berrien,1,0.000000,0.000000,-0.666667,1,1
7,Bleckley,1,-0.666667,0.000000,0.000000,1,1
8,Brantley,1,0.400000,0.000000,2.500000,7,1
9,Brooks,1,0.000000,-0.500000,0.000000,1,1
